In [ ]:
!pip install requests

In [ ]:
# import libraries + set API key + headers
import requests
import json
import pandas as pd
from pandas import json_normalize
from collections import defaultdict
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity
from geopy.distance import geodesic
from sklearn.preprocessing import LabelEncoder
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MultiLabelBinarizer

api_key = "UHj90QwExx5BbtaOh-Ye2gI7vN_0bF1NQ1fgxlSrBIvfFyvkH-ObKKzLMpbTMA8aYaaXhY_6bHMXzxOfJ1IZyrUGwvubJRkRSiX5eLCWrIXwXEiDVn4nZGb3aVBCZ3Yx"
headers = {"Authorization": f"Bearer {api_key}"}

In [ ]:
# make search request
url = "https://api.yelp.com/v3/businesses/search"
params = {
    "categories": "restaurants",
    "location": "Manhattan",
}

all_data = [] # list of dictionaries, each dictionary is a business
for offset in range(0, 240, 20):  # fetch 20 results in each request, up to 240 results
    params["offset"] = offset
    response = requests.get(url, headers=headers, params=params)

    # check status code of response
    if response.status_code == 200:
        data = response.json()
        all_data.extend(data.get("businesses", []))

        if len(data.get("businesses", [])) < 20:
            print(f"No more results at offset {offset}.")
            break
    else:
        print(f"Request failed at offset {offset} with status code: {response.status_code}")
        print(f"Response message: {response.text})")
        break

print(json.dumps(all_data, indent=2))

[
  {
    "id": "z5hRX3iJ5Ty_S38iG_WY3Q",
    "alias": "tanner-smiths-new-york-2",
    "name": "Tanner Smiths",
    "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/glqvzA2jZTi3nLYg7hvEKw/o.jpg",
    "is_closed": false,
    "url": "https://www.yelp.com/biz/tanner-smiths-new-york-2?adjust_creative=F_flGeunYcXBgkCZ1oO11Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=F_flGeunYcXBgkCZ1oO11Q",
    "review_count": 2235,
    "categories": [
      {
        "alias": "cocktailbars",
        "title": "Cocktail Bars"
      },
      {
        "alias": "newamerican",
        "title": "New American"
      },
      {
        "alias": "beerbar",
        "title": "Beer Bar"
      }
    ],
    "rating": 4.3,
    "coordinates": {
      "latitude": 40.76442,
      "longitude": -73.98169
    },
    "transactions": [
      "pickup",
      "delivery"
    ],
    "price": "$$",
    "location": {
      "address1": "204 W 55th St",
      "address2": "",
      "address3": "",
      "c

In [ ]:
# convert all_data into a dataframe
df = pd.json_normalize(all_data, sep="_")

# select only the specified columns
selected_columns = ['name', 'rating', 'price', 'categories', 'coordinates_latitude', 'coordinates_longitude']
df = df[selected_columns]

# extract titles from categories column
def extract_titles(categories):
    return ', '.join([category['title'] for category in categories])
df['categories'] = df['categories'].apply(extract_titles)

# clean data
df = df.dropna()
df = df.drop_duplicates()

# display the DataFrame
pd.set_option('display.max_rows', None)
df

,name,rating,price,categories,coordinates_latitude,coordinates_longitude
0,Tanner Smiths,4.3,$$,"Cocktail Bars, New American, Beer Bar",40.764420,-73.981690
1,Nerai,4.4,$$$,"Greek, Seafood, Mediterranean",40.760265,-73.973089
2,La Grande Boucherie,4.4,$$$,"French, Steakhouses, Cocktail Bars",40.762627,-73.980841
3,Haswell Green's,4.5,$$,"New American, Piano Bars, Sports Bars",40.763010,-73.984160
4,Her Name Is Han,4.3,$$,"Korean, Breakfast & Brunch",40.746262,-73.984734
5,The Smith,4.0,$$$,"New American, Breakfast & Brunch, Cocktail Bars",40.755165,-73.967868
6,Robert,4.0,$$$,"New American, Venues & Event Spaces, Cocktail ...",40.767394,-73.982036
7,212 Steakhouse,4.0,$$$,"Steakhouses, Seafood, Halal",40.756340,-73.966489
8,Mission Ceviche Upper East Side,4.4,$$$$,"Peruvian, Seafood, Cocktail Bars",40.769233,-73.957876
9,American Brass,4.3,$$$,"Seafood, Cocktail Bars, New American",40.743971,-73.958493


In [ ]:
# extract first restaurant from dataframe
df_all_data_0 = pd.json_normalize(all_data[0], sep="_")
selected_columns = ['name', 'rating', 'price', 'categories', 'coordinates_latitude', 'coordinates_longitude']
df_all_data_0 = df_all_data_0[selected_columns]

# extract titles from categories column
def extract_titles(categories):
    return ', '.join([category['title'] for category in categories])
df_all_data_0['categories'] = df_all_data_0['categories'].apply(extract_titles)
df_all_data_0

,name,rating,price,categories,coordinates_latitude,coordinates_longitude
0,Tanner Smiths,4.3,$$,"Cocktail Bars, New American, Beer Bar",40.76442,-73.98169


# Markov Chain

In [ ]:
# Markov chain
# example list of cuisines the user has visited
visited_cuisines = ["New American", "Korean", "Italian", "New American", "Korean", "Izakaya", "New American", "Middle Eastern", "Italian", "New American"]

In [ ]:
# construct transition matrix
def build_transition_matrix(cuisines):
    transition_matrix = defaultdict(lambda: defaultdict(int))

    # create pairs of transitions (current_cuisine -> next_cuisine)
    for i in range(len(cuisines) - 1):
        current = cuisines[i]
        next_cuisine = cuisines[i + 1]
        transition_matrix[current][next_cuisine] += 1

    # normalize matrix to get probabilities
    for current in transition_matrix:
        total = sum(transition_matrix[current].values())
        for next_cuisine in transition_matrix[current]:
            transition_matrix[current][next_cuisine] /= total

    return transition_matrix

# build transition matrix from this sequence
transition_matrix = build_transition_matrix(visited_cuisines)

# convert transition matrix to a dataframe
transition_df = pd.DataFrame.from_dict(transition_matrix, orient='index').fillna(0)
print("Transition Matrix:")
transition_df


Transition Matrix:


,Korean,Middle Eastern,Italian,Izakaya,New American
New American,0.666667,0.333333,0.0,0.0,0.0
Korean,0.000000,0.000000,0.5,0.5,0.0
Middle Eastern,0.000000,0.000000,1.0,0.0,0.0
Italian,0.000000,0.000000,0.0,0.0,1.0
Izakaya,0.000000,0.000000,0.0,0.0,1.0


In [ ]:
# simulate next cuisine
def predict_next_cuisine(current_cuisine, transition_matrix):
    if current_cuisine not in transition_matrix:
        print("No transitions available for this cuisine.")
        return None

    # get possible next cuisines and their probabilities
    next_cuisines = list(transition_matrix[current_cuisine].keys())
    probabilities = list(transition_matrix[current_cuisine].values())

    # randomly select next cuisine based on probabilities
    next_cuisine = random.choices(next_cuisines, probabilities)[0]
    return next_cuisine

# predict next cuisine after the last visit
last_visited_cuisine = visited_cuisines[-1]
predicted_cuisine = predict_next_cuisine(last_visited_cuisine, transition_matrix)

print(f"The user is most likely to visit a {predicted_cuisine} restaurant next.")

The user is most likely to visit a Korean restaurant next.


In [ ]:
# filter rows where 'categories' column contains predicted cuisine
df_pref_cuisine = df[df['categories'].str.contains(predicted_cuisine, case=False, na=False)]

# display filtered dataframe
df_pref_cuisine.head(3)

,name,rating,price,categories,coordinates_latitude,coordinates_longitude
4,Her Name Is Han,4.3,$$,"Korean, Breakfast & Brunch",40.746262,-73.984734
13,Osamil,4.3,$$$,"Korean, Gastropubs, Tapas/Small Plates",40.746970,-73.986400
17,Mari,4.3,$$$$,Korean,40.761520,-73.990610


# Content-based Filtering
The distance_from_user column represents the calculated Haversine distance (in meters) from the user's average location to the restaurant, and the final_score column is a weighted score combining location and price similarities.

In [ ]:
# sample user data
data = {
    'name': ["Friedman's", "Anytime Kitchen", "Scarpetta", "The Smith", "Osamil", "Izakaya MEW", "Oscar Wilde", "Ravagh Persion Grill", "Ulivo", "John Doe"],
    'rating': [4.0, 4.6, 4.1, 4.1, 4.3, 4.3, 3.8, 4.0, 4.0, 4.0],
    'price': ['$$', '$$', '$$$', '$$', '$$$', '$$', '$$', '$$', '$$', '$$'],
    'categories': ['New American, Breakfast & Brunch, Wine Bars', 'Korean, Tapas/Small Plates, Barbeque', 'Italian, Bars, Desserts', 'New American, Breakfast & Brunch, Cocktail Bars', 'Korean, Gastropubs, Tapas/Small Plates', 'Tapas Bars, Sushi Bars, Izakaya', 'Cocktail Bars, New American, Gastropubs', 'Middle Eastern, Persian/Iranian, Mediterranean', 'Italian, Cocktail Bars', 'New American, Gastropubs'],
    'coordinates_latitude': [40.748647, 40.747763, 40.744768, 40.744311, 40.746970	, 40.750148, 40.745269, 40.747309, 40.744982, 40.744916],
    'coordinates_longitude': [-73.990865, -73.986779, -73.985555, -73.988741, -73.986400, -73.986320, -73.990018, -73.983126, -73.987969, -73.986868]
}

user_visits_df = pd.DataFrame(data)
user_visits_df

,name,rating,price,categories,coordinates_latitude,coordinates_longitude
0,Friedman's,4.0,$$,"New American, Breakfast & Brunch, Wine Bars",40.748647,-73.990865
1,Anytime Kitchen,4.6,$$,"Korean, Tapas/Small Plates, Barbeque",40.747763,-73.986779
2,Scarpetta,4.1,$$$,"Italian, Bars, Desserts",40.744768,-73.985555
3,The Smith,4.1,$$,"New American, Breakfast & Brunch, Cocktail Bars",40.744311,-73.988741
4,Osamil,4.3,$$$,"Korean, Gastropubs, Tapas/Small Plates",40.746970,-73.986400
5,Izakaya MEW,4.3,$$,"Tapas Bars, Sushi Bars, Izakaya",40.750148,-73.986320
6,Oscar Wilde,3.8,$$,"Cocktail Bars, New American, Gastropubs",40.745269,-73.990018
7,Ravagh Persion Grill,4.0,$$,"Middle Eastern, Persian/Iranian, Mediterranean",40.747309,-73.983126
8,Ulivo,4.0,$$,"Italian, Cocktail Bars",40.744982,-73.987969
9,John Doe,4.0,$$,"New American, Gastropubs",40.744916,-73.986868


In [ ]:
# considered: price, location
# encode price column
price_encoder = LabelEncoder()
df['price_encoded'] = price_encoder.fit_transform(df['price'])
user_visits_df['price_encoded'] = price_encoder.transform(user_visits_df['price'])

# calculate location similarity using Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on the earth's surface
    given their latitude and longitude in decimal degrees.
    """
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geodesic(coords_1, coords_2).meters

# calculate average latitude and longitude of user visits to represent location
user_avg_location = user_visits_df[['coordinates_latitude', 'coordinates_longitude']].mean()

# create column for distance from user to restaurant
df['distance_from_user'] = df.apply(
    lambda row: haversine_distance(user_avg_location['coordinates_latitude'], \
                                   user_avg_location['coordinates_longitude'], \
                                   row['coordinates_latitude'], \
                                   row['coordinates_longitude']),
    axis=1
)

# calculate cosine similarity for price using average price encoding from user visits
user_avg_price_encoded = user_visits_df['price_encoded'].mean()

# reshape user_avg_price_encoded to match shape of df['price_encoded']
user_avg_price_encoded_reshaped = np.array([user_avg_price_encoded]).reshape(1, -1)

# compute cosine similarity between average user price and restaurant price encodings
df['price_similarity'] = cosine_similarity(
    user_avg_price_encoded_reshaped,
    df['price_encoded'].values.reshape(-1, 1)
).flatten()

# combine location and price similarities into a final score
location_weight = 0.5  # weight for location similarity
price_weight = 0.5  # weight for price similarity

df['final_score'] = (
    location_weight * (1 / (df['distance_from_user'] + 1)) +  # We inverse the distance to make closer places higher
    price_weight * df['price_similarity']
)

# sort restaurants by final score
recommended_restaurants = df.sort_values(by='final_score', ascending=False)

# recommend top n restaurants
top_n = 3
recommended_restaurants = recommended_restaurants[~recommended_restaurants['name'].isin(user_visits_df['name'])]

# output the recommended restaurants
recommended_restaurants_top_n = recommended_restaurants.head(top_n)
recommended_restaurants_top_n[['name', 'price', 'distance_from_user', 'final_score']]

,name,price,distance_from_user,final_score
60,Woorijip,$$,125.661085,0.503948
118,Turntable LP Bar & Karaoke,$$,131.427014,0.503776
68,Gaonnuri,$$$$,134.075245,0.503702


In [ ]:
# display final restaurant recommendations to user
print("Go to these", predicted_cuisine, "restaurants next!")
for name in df_pref_cuisine['name']:
    print(f"{name:<50}")
print("\n")
print("We think you'll like these restaurants based on your location and price preferences:")
for name in recommended_restaurants_top_n['name']:
  print(f"{name:<50}")

Go to these Korean restaurants next!
Her Name Is Han                                   
Osamil                                            
Mari                                              
LOVE Korean BBQ                                   
Yoon Haeundae Galbi                               
Atoboy                                            
NARO                                              
Take31                                            
JUA                                               
Olle                                              
Woorijip                                          
Gaonnuri                                          
Oiji Mi                                           
Rib No 7                                          
BCD Tofu House                                    
Soju Haus                                         
The Jin                                           
Antoya Korean BBQ                                 
Turntable LP Bar & Karaoke                   

# SVD

In [ ]:
# sample data
# simulated user-restaurant interaction matrix
# rows are users, columns are restaurants, values are interaction scores (how many times users go to specific resturant)


# sample dining history of 5 users interacting with restaurants from Yelp API
sample_data = {
    "User1": {  # prefers Italian and mid-priced ($$) restaurants
        "Quality Italian": 5,
        "Allora Ristorante": 4,
        "Lincoln Ristorante": 4,
        "Barolo East": 5,
        "Pazza Notte": 3,
        "Nino's 46": 4,
        "Cafe Fiorello": 5,
        "Olio e Più": 3,
        "Tony's Di Napoli": 2,
        "Turkuaz Restaurant": 1,  # minor outlier
    },
    "User2": {  # prefers Japanese cuisine and higher-priced ($$$ or $$$$) restaurants
        "Ootoya Times Square": 5,
        "omakase room by maaser": 5,
        "Tatsuda Omakase": 5,
        "Sushi Yasuda": 4,
        "Trattoria Dell'Arte": 2,
        "Jun-Men Ramen Bar": 5,
        "Sushi You": 4,
        "Koi Restaurant": 3,
        "Toribro Ramen": 5,
        "Gu Japanese Fusion Sushi & Bar": 4,
    },
    "User3": {  # prefers Italian and French cuisines at mid-range ($$) restaurants
        "Nino's 46": 4,
        "IL Punto Ristorante": 3,
        "Barolo East": 5,
        "Pazza Notte": 4,
        "Amali": 4,
        "Ai Fiori": 5,
        "Cafe Fiorello": 4,
        "Follia": 3,
        "Aunt Jake's": 4,
        "Gramercy Tavern": 4,
    },
    "User4": {  # prefers American and fusion cuisines at moderately priced ($$$) restaurants
        "The Smith": 4,
        "SoleLuna Restaurant": 4,
        "Gallaghers Steakhouse": 4,
        "Cyclo": 3,
        "Anassa Taverna Astoria": 3,
        "Tacombi": 3,
        "Birdland": 4,
        "Emmy Squared Pizza - Upper East Side": 3,
        "The Jeffrey Craft Beer & Bites": 3,
        "Uva Next Doo": 3,
    },
    "User5": {  # prefers pizza and Mediterranean cuisine at budget-friendly ($$) places
        "Olio e Più": 5,
        "Angelo's Coal Oven Pizzeria": 5,
        "VEZZO NYCThinCrust Pizza": 5,
        "Follia": 4,
        "Allora Ristorante": 4,
        "Pippali": 3,
        "Hav & Mar": 3,
        "Lena's Italian Kitchen": 4,
        "Traif": 2,
        "Wine 30": 3,
    },
}

# create interaction matrix
# extract unique restaurant names
restaurant_names = df['name'].unique()
interaction_matrix = pd.DataFrame(0, index=sample_data.keys(), columns=restaurant_names)

# populate the matrix with user dining history
for user, visits in sample_data.items():
    for restaurant, frequency in visits.items():
        if restaurant in interaction_matrix.columns:
            interaction_matrix.loc[user, restaurant] = frequency

print("Original Interaction Matrix (User-Restaurant Frequencies):")
print(interaction_matrix)


In [ ]:
# apply SVD
# decompose matrix into U, Sigma, VT
interaction_matrix_np = interaction_matrix.to_numpy(dtype=float)

# apply SVD
k = 2  # number of latent features
U, sigma, VT = svds(interaction_matrix_np, k=k)
sigma = np.diag(sigma)

# reconstruct matrix
reconstructed_matrix = np.dot(np.dot(U, sigma), VT)
reconstructed_df = pd.DataFrame(reconstructed_matrix, index=interaction_matrix.index, columns=interaction_matrix.columns)

print("\nReconstructed Interaction Matrix:")
print(reconstructed_df)

# recommend restaurants
def recommend_restaurants(user_id, interaction_matrix, reconstructed_df, df, top_n=3):
    """
    Generate top N restaurant recommendations for a given user.
    """
    # get restaurants user has already interacted with
    known_interactions = interaction_matrix.loc[user_id]
    known_restaurants = known_interactions[known_interactions > 0].index.tolist()

    # get predicted scores and sort in descending order
    user_predictions = reconstructed_df.loc[user_id].sort_values(ascending=False)

    # exclude restaurants user already knows
    recommendations = user_predictions[~user_predictions.index.isin(known_restaurants)].head(top_n)

    # check if recommendations are empty
    if recommendations.empty:
        print(f"No new recommendations for {user_id}")
        return pd.DataFrame(columns=df.columns)

    # retrieve restaurant details from Yelp dataset
    recommended_restaurants = df[df['name'].isin(recommendations.index)].copy()
    recommended_restaurants['predicted_score'] = recommended_restaurants['name'].map(recommendations)
    return recommended_restaurants

# generate recommendations for each user
for user in interaction_matrix.index:
    recommended_restaurants_df = recommend_restaurants(user, interaction_matrix, reconstructed_df, df)
    print(f"\nTop Recommendations for {user}:")
    display(recommended_restaurants_df)
